<a href="https://colab.research.google.com/github/KOFIYEB/Chatbox_Zephyr7B/blob/main/chatbox_Zephyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbox with Zephyr 7B**

---



We will develop a chatbot using **Zephyr 7B**, a well-regarded conversational model that has been enhanced through fine-tuning with Mistral 7B. Crafted by the Hugging Face H4 team, Zephyr 7B Beta has demonstrated superior performance compared to GPT-3.5, LLama 2 70B, and other models in the 7B size category, as evidenced by the Alpaca Leaderboard rankings. Additionally, Zephyr 7B offers the advantage of being significantly more compact, requiring only 4% of the storage space needed for GPT-3.5.

# **Load Packages**

In [ ]:
# You only need to run this once, even if you re-start the kernel or machine
# Remove the -q to print install output
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 14.6 MB/s eta 0:00:00


# **Load Model**

In this section, we initialize the model. You're welcome to substitute it with any conversational or instructional model from Hugging Face, ensuring that you adjust the "roles" accordingly as outlined below. However, this model is for now highly recommended.

In [ ]:
models_dict = {
    "Zephyr 7B Alpha": "HuggingFaceH4/zephyr-7b-alpha",
    "Zephyr 7B Beta": "HuggingFaceH4/zephyr-7b-beta",
    # You can add more chat/instruct models here
}

model_id = models_dict["Zephyr 7B Beta"]

In [ ]:
import torch
from transformers import pipeline

# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# **Define the role**

specify the role you want the chatbot to adopt. For this example, I'll configure it to function as a sports analyst.

In [ ]:
job_description = "You are a sport analyst expert. Please answer my questions."

Here we execute the model! To conclude the question-answering session, type "stop" or "exit". Feel free to disregard the UserWarning about changes to the pre-trained model configuration.

In [ ]:
messages = [
    {
        "role": "system",
        "content": job_description,
    },
]

exit_terms = ["stop", "exit"]

while True:
    question = input("\nQuestion:")
    if question.lower() in exit_terms:
        break
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    print(output[response_start + len('<|assistant|>'):])


Question:Rank the top 10 basketball players of all time


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



1. Michael Jordan: Widely regarded as the greatest basketball player of all time, Jordan won six NBA championships and was a 14-time NBA All-Star.

2. LeBron James: Four-time NBA champion, four-time NBA MVP, and a 17-time NBA All-Star, James is considered one of the most dominant players in NBA history.

3. Kareem Abdul-Jabbar: Six-time NBA champion and six-time NBA MVP, Abdul-Jabbar holds the all-time NBA scoring record with 38,387 points.

4. Bill Russell: Eleven-time NBA champion and five-time NBA MVP, Russell is the most successful player in NBA history in terms of championships.

5. Magic Johnson: Three-time NBA champion and three-time NBA MVP, Johnson was a dominant point guard in the 1980s.

6. Wilt Chamberlain: Four-time NBA champion and one-time NBA MVP, Chamberlain holds numerous NBA records, including most points in a single game (100).

7. Larry Bird: Three-time NBA champion and three-time NBA MVP, Bird was a master of the mid-range jumper and led the Boston Celtics to an 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Lebron James has already achieved some of the same accolades as Michael Jordan, such as four NBA championships and four NBA MVP awards. However, to surpass Jordan as the greatest basketball player of all time, James would need to accomplish some of the following:

1. Win more NBA championships: Jordan won six NBA championships in his career, and while James has four, he would need to win two more to surpass Jordan's total.

2. Win more NBA Finals MVP awards: Jordan won the Finals MVP award six times, while James has won it four times. James would need to win another two Finals MVP awards to tie Jordan's total.

3. Achieve better statistical marks: While James has already passed Jordan in some statistical categories, such as career points and assists, there are still some marks that Jordan holds that James has yet to surpass. For example, Jordan holds the NBA record for career playoff points, and James would need to break that record to surpass Jordan in that category.

4. Achieve grea

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



I did consider those statistical categories in my previous answer, and I mentioned that while Lebron James has already passed Michael Jordan in some statistical categories, such as career points and assists, there are still some marks that Jordan holds that James has yet to surpass, like the NBA record for career playoff points.

Regarding the number of Finals reached, both Jordan and James have appeared in the NBA Finals multiple times. Jordan appeared in the Finals a total of 6 times and won 6 championships, while James has appeared in the Finals a total of 9 times and won 4 championships. While James has reached the Finals more times than Jordan, Jordan's winning percentage in the Finals (100%) is higher than James' (67%).

Overall, I believe that while statistics are certainly an important factor to consider when evaluating a player's career, there are other factors that need to be taken into account as well, such as the player's impact on the game, their leadership abilities, and

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



It's difficult to predict which current player will make the top 10 list of all time, as the game and the league are constantly evolving, and there are many talented players in the NBA today. However, some current players who have a strong chance of making the list include:

1. Stephen Curry: The two-time NBA MVP and three-time NBA champion has already achieved a lot in his career, and he's still in his prime. Curry's incredible shooting ability and leadership skills have helped him lead the Golden State Warriors to multiple championships, and he's on track to break several NBA records.

2. Kevin Durant: A four-time NBA scoring champion and two-time NBA champion, Durant is one of the most talented scorers in NBA history. His versatility and ability to score from anywhere on the court have made him one of the most feared players in the league, and he's still in his prime.

3. Giannis Antetokounmpo: The reigning NBA MVP and two-time NBA champion has already achieved a lot in his career,

**It worked absolutely fine, you can also create your version of ChatGPT 😲**